In [12]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import numpy as np

In [ ]:
logits = tf.random.uniform(minval=0, maxval=1, seed=42, shape=(5, 6))
logits

In [ ]:
Q_marginal = tfd.MixtureSameFamily(
mixture_distribution=tfd.Categorical(logits=tf.zeros(tf.shape(logits)[0])),
    components_distribution=tfd.RelaxedBernoulli(logits=tf.transpose(logits),
                                                 temperature=1e-5)
)
Q_marginal

In [ ]:
x = Q_marginal.sample(512)
x = tf.clip_by_value(x, 1e-7, 1.-1e-7)
tf.reduce_mean(-1. * tf.reduce_sum(Q_marginal.log_prob(x), axis=1))


In [ ]:
tf.reduce_mean(-1. * Q_marginal.log_prob(x))

In [ ]:
tfd.RelaxedBernoulli(logits=logits[0], temperature=1e-5).sample(5)

In [ ]:
tf.reshape(tf.repeat(tf.one_hot(tf.range(5), 5), 4, axis=0), shape=(5, 4, 5))

In [ ]:
logits = tf.random.uniform(shape=(4, 6))

In [ ]:
tf.stack([logits] * 5)

In [ ]:
tf.stack([tf.one_hot(tf.range(5), 5)] * 5)

In [ ]:
tf.reshape(tf.repeat(tf.expand_dims(tf.one_hot(2, 5), axis=0), 3, axis=0), shape=(3, 5))

In [ ]:
tf.stack([tf.one_hot(2, 5)] * 3)

In [ ]:
latent_state_size = 3
batch_size = 6
action_size = 2

action = tf.random.uniform(
    shape=(batch_size, ), minval=0, maxval=action_size, dtype=tf.int32, seed=42)
action = tf.one_hot(action, action_size)

latent_states_logits = []
for _ in range(action_size):
    latent_states_logits.append(
        tf.random.uniform(shape=(batch_size, latent_state_size), seed=42))
# latent_states_stack = tf.stack(latent_states_logits, axis=1)

latent_states_stack = tf.random.uniform(shape=(batch_size, action_size, latent_state_size))
action_expand_dims = tf.expand_dims(action, -1)
action_tile = tf.tile(action_expand_dims, (1, 1, latent_state_size))
mult = action_tile * latent_states_stack
latent_states = tf.reduce_sum(mult, axis=1)

print("action", action)
print("latent_states_stack", latent_states_stack)
print("action_tile", action_tile)
print("mult", mult)
print(latent_states)

In [ ]:
print(tf.tile(action_expand_dims, (1, 1, latent_state_size)))
print(tf.reshape(tf.repeat(action_expand_dims, latent_state_size), shape=(batch_size, action_size, latent_state_size)))
print(tf.repeat(action_expand_dims, latent_state_size, axis=-1))

In [ ]:
layers = []
# layers.append(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, -1)))
layers.append(tf.keras.layers.RepeatVector(latent_state_size))
layers.append(tf.keras.layers.Permute(dims=(2, 1)))
sequential = tf.keras.Sequential(layers)
sequential(action)

In [ ]:
labels_logits = tf.random.uniform(shape=(8, 3))
latent_states_logits = tf.random.uniform(shape=(8, 7))
joint_distribution = tfd.JointDistributionSequential(
    [
        tfd.Independent(tfd.Bernoulli(logits=labels_logits, dtype=tf.float32)),
        lambda l: tfd.Independent(
            tfd.Logistic(loc=tf.concat([latent_states_logits, l], axis=-1) / 0.5,
                               scale=1. / 0.5))
    ], validate_args=True
)
samples = joint_distribution.sample()
samples

In [ ]:
next_label = tf.round(tf.random.uniform(shape=(8, 3), maxval=1, dtype=tf.float32))
next_latent_state = tf.random.uniform(shape=(8, 10))
next_label, next_latent_state

In [ ]:
joint_distribution.log_prob(samples)

In [ ]:
joint_distribution.log_prob_parts(samples)

In [ ]:
tfd.Independent(tfd.Bernoulli(logits=labels_logits, dtype=tf.float32)).kl_divergence(
    tfd.Independent(tfd.Bernoulli(logits=labels_logits/3., dtype=tf.float32)),
)

In [ ]:
d = tfd.Independent(tfd.Bernoulli(logits=labels_logits, dtype=tf.float32))
d.log_prob(samples[0])

In [ ]:
latent_states_logits = tf.random.uniform(shape=(2, 5))
print(latent_states_logits)
tf.tile(tf.expand_dims(latent_states_logits, axis=0), [2, 1, 1])

In [ ]:
logits = tf.random.uniform(shape=(8, 5, 2))
print(logits)
print(tf.transpose(logits, perm=[0, 2, 1]))

In [ ]:
tfd.Independent(tfd.Bernoulli(logits=tf.transpose(logits, perm=[0, 2, 1])), reinterpreted_batch_ndims=1)

In [ ]:
np.unpackbits(np.array([[1, 3, 5, 256]], np.uint8)).reshape(-1, 16)

In [11]:
x = tf.range(2**12)
latent_state_size = 12
tf.map_fn(lambda n: (n // 2 ** tf.range(latent_state_size)) % 2, x)
# print(tf.reduce_sum(_x * 2**tf.range(12), axis=-1))

<tf.Tensor: shape=(4096, 12), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>

In [16]:
states = tf.constant([3, 3, 2, 2, 2, 2, 4, 3, 3])
actions = tf.constant([0, 0, 0, 0, 0, 1, 1, 1, 1])
next_states = tf.constant([1, 1, 2, 1, 3, 4, 5, 5, 3])
print(states, actions, next_states)

tf.Tensor([3 3 2 2 2 2 4 3 3], shape=(9,), dtype=int32) tf.Tensor([0 0 0 0 0 1 1 1 1], shape=(9,), dtype=int32) tf.Tensor([1 1 2 1 3 4 5 5 3], shape=(9,), dtype=int32)


In [29]:
num_states = 6
num_actions = 2

transitions = (states * num_actions * num_states +  # index of the state group
               actions * num_states +  # index of the action group
               next_states)  # index of the next state
transitions, _, count = tf.unique_with_counts(transitions)

print(transitions)
print(count)

tf.Tensor([37 26 25 27 34 59 47 45], shape=(8,), dtype=int32)
tf.Tensor([2 1 1 1 1 1 1 1], shape=(8,), dtype=int32)


In [30]:
new_transitions = tf.stack([transitions // (num_states * num_actions), transitions % num_states], axis=-1)
print(transitions)

tf.Tensor([37 26 25 27 34 59 47 45], shape=(8,), dtype=int32)


In [ ]:
sparse_transitions = tf.sparse.SparseTensor(
    indices=transitions,
    values=count,
    dense_shape=(num_states * num_actions, num_states)
)